# Organize data for running PCA

Two main approaches:
1. select a subset of all ontogeny sessions, a few sessions from each age and sex
2. select a subset of all longtogeny sessions, a few sessions from each age and sex. Make sure to have the same number of sessions and similar ages per mouse.

In [1]:
import json
import h5py
import pandas as pd
from pathlib import Path
from toolz import concat, curry, frequencies
from aging.organization.paths import FOLDERS
from aging.organization.dataframes import get_experiment, get_age

In [2]:
FOLDERS

(PosixPath('/n/groups/datta/Dana/Ontogeny/raw_data/Ontogeny_females'),
 PosixPath('/n/groups/datta/Dana/Ontogeny/raw_data/Ontogeny_males'),
 PosixPath('/n/groups/datta/Dana/Ontogeny/raw_data/Dana_ontogeny/Males'),
 PosixPath('/n/groups/datta/Dana/Ontogeny/raw_data/Dana_ontogeny/Females'),
 PosixPath('/n/groups/datta/Dana/Ontogeny/raw_data/longtogeny_pre_unet/Males'),
 PosixPath('/n/groups/datta/min/longtogeny_072023/Males'),
 PosixPath('/n/groups/datta/min/longtogeny_072023/Females'),
 PosixPath('/n/groups/datta/min/wheel_062023'),
 PosixPath('/n/groups/datta/win/longtogeny/dlight'),
 PosixPath('/n/groups/datta/min/longtogeny_052023/Males'),
 PosixPath('/n/groups/datta/win/longtogeny/data/jackson-labs/datta_i'))

In [3]:
arhmm_version = 12
size_norm_key = 'win_size_norm_frames_v6'

In [4]:
session_exclusion_list = '/n/groups/datta/win/longtogeny/data/metadata/Ontogeny experiment list - Session quality control.csv'
exclusion_df = pd.read_csv(session_exclusion_list, header=0)

In [5]:
exclusion_df['Keep?'] = exclusion_df['Keep?'].str.lower()
exclusion_df['Keep?'].unique()

array(['y', 'n', '?'], dtype=object)

In [6]:
exclusions = exclusion_df.query('`Keep?` != "y"')

In [8]:
def is_size_normalized(file):
    try:
        with h5py.File(file, 'r') as h5f:
            return size_norm_key in h5f
    except OSError:
        print(file)
        return False


def not_excluded(file):
    return str(file) not in exclusions['File path'].values


@curry
def thread_and(funcs, x):
    return all(f(x) for f in funcs)


@curry
def not_double(parent_counts, path):
    if "jackson" not in str(path) and parent_counts[str(path.parents[1])] > 1:
        return 'proc_cleaned' == path.parent.name
    return True

## Organize ontogeny sessions

These are the data that will be run through PCA and the modeling steps

- create a dataframe with the file path, age, sex, and session name
- do a groupby and subsample the same number of sessions for each (age, sex) group
- symlink results.h5 files to a new folder

In [9]:
agg_folder = Path(f'/n/groups/datta/win/longtogeny/data/ontogeny/version_{arhmm_version:02d}/training_data')
agg_folder.mkdir(exist_ok=True, parents=True)

In [11]:
FOLDERS[:4]

(PosixPath('/n/groups/datta/Dana/Ontogeny/raw_data/Ontogeny_females'),
 PosixPath('/n/groups/datta/Dana/Ontogeny/raw_data/Ontogeny_males'),
 PosixPath('/n/groups/datta/Dana/Ontogeny/raw_data/Dana_ontogeny/Males'),
 PosixPath('/n/groups/datta/Dana/Ontogeny/raw_data/Dana_ontogeny/Females'))

In [12]:
# filter files for the ones with size_normalized_frames
files = sorted(
    filter(
        thread_and((is_size_normalized, not_excluded)),
        concat(f.glob("**/*results_00.h5") for f in FOLDERS[:4]),
    )
)

/n/groups/datta/Dana/Ontogeny/raw_data/Dana_ontogeny/Males/12months_28042021/session_20210428102836/proc_cleaned/results_00.h5


In [13]:
parent_counts = frequencies(str(f.parents[1]) for f in files)
files = list(filter(not_double(parent_counts), files))

In [14]:
df = []
for f in files:
    try:
        md = json.load((f.parents[1] / "metadata.json").open())
        data = dict(
            age=get_age(f),
            experiment=get_experiment(f),
            session=f.parents[1].name,
            # sex=f.parents[3].name.split("_")[-1][:-1],
            path=str(f),
            SessionName=md["SessionName"],
        )
        df.append(data)
    except KeyError:
        print("KeyError: ", f)
    except FileNotFoundError:
        continue
df = pd.DataFrame(df)

KeyError:  /n/groups/datta/Dana/Ontogeny/raw_data/Dana_ontogeny/Females/session_20230721072116/proc/results_00.h5
KeyError:  /n/groups/datta/Dana/Ontogeny/raw_data/Dana_ontogeny/Females/session_20230721072215/proc/results_00.h5
KeyError:  /n/groups/datta/Dana/Ontogeny/raw_data/Dana_ontogeny/Females/session_20230721072329/proc/results_00.h5
KeyError:  /n/groups/datta/Dana/Ontogeny/raw_data/Dana_ontogeny/Females/session_20230721082639/proc/results_00.h5
KeyError:  /n/groups/datta/Dana/Ontogeny/raw_data/Dana_ontogeny/Females/session_20230721082719/proc/results_00.h5
KeyError:  /n/groups/datta/Dana/Ontogeny/raw_data/Dana_ontogeny/Females/session_20230721082802/proc/results_00.h5
KeyError:  /n/groups/datta/Dana/Ontogeny/raw_data/Dana_ontogeny/Females/session_20230721091042/proc/results_00.h5
KeyError:  /n/groups/datta/Dana/Ontogeny/raw_data/Dana_ontogeny/Females/session_20230721091057/proc/results_00.h5
KeyError:  /n/groups/datta/Dana/Ontogeny/raw_data/Dana_ontogeny/Females/session_20230721

In [15]:
len(df)

905

In [16]:
df.head()

,age,experiment,session,path,SessionName
0,52,dana_ontogeny_females,session_20221018094245,/n/groups/datta/Dana/Ontogeny/raw_data/Dana_on...,fo_12months
1,52,dana_ontogeny_females,session_20221018094429,/n/groups/datta/Dana/Ontogeny/raw_data/Dana_on...,fo_12months
2,52,dana_ontogeny_females,session_20221018094537,/n/groups/datta/Dana/Ontogeny/raw_data/Dana_on...,fo_12months
3,52,dana_ontogeny_females,session_20221018094629,/n/groups/datta/Dana/Ontogeny/raw_data/Dana_on...,fo_12months
4,52,dana_ontogeny_females,session_20221018110503,/n/groups/datta/Dana/Ontogeny/raw_data/Dana_on...,fo_12months


In [17]:
df = df[~df['SessionName'].str.contains("CRL")]

In [18]:
df.groupby(['age', 'experiment']).count()

session  path  SessionName
age experiment                                       
3   dana_ontogeny_females       24    24           24
    dana_ontogeny_males         15    15           15
5   dana_ontogeny_females       24    24           24
    dana_ontogeny_males         16    16           16
7   dana_ontogeny_females       24    24           24
    dana_ontogeny_males         16    16           16
9   dana_ontogeny_females       24    24           24
    dana_ontogeny_males         16    16           16
12  dana_ontogeny_females       24    24           24
    dana_ontogeny_males         27    27           27
16  dana_ontogeny_females       16    16           16
    ontogeny_males              16    16           16
20  ontogeny_females            16    16           16
    ontogeny_males              16    16           16
24  dana_ontogeny_females       24    24           24
    dana_ontogeny_males         24    24           24
28  ontogeny_males              16    16           16
32  ontogeny_females            16    16           16
    ontogeny_males              16    16           16
35  dana_ontogeny_females       16    16           16
    dana_ontogeny_males         16    16           16
36  ontogeny_females            16    16           16
    ontogeny_males              16    16           16
40  ontogeny_females            16    16           16
    ontogeny_males              16    16           16
44  ontogeny_females            16    16           16
    ontogeny_males              16    16           16
48  ontogeny_females            16    16           16
    ontogeny_males              16    16           16
52  dana_ontogeny_females       24    24           24
    dana_ontogeny_males         26    26           26
56  ontogeny_females            16    16           16
    ontogeny_males              16    16           16
60  ontogeny_females            16    16           16
    ontogeny_males              16    16           16
64  ontogeny_females            16    16           16
    ontogeny_males              15    15           15
68  ontogeny_females            16    16           16
    ontogeny_males              16    16           16
72  ontogeny_males              16    16           16
78  dana_ontogeny_females       24    24           24
    dana_ontogeny_males         27    27           27
    ontogeny_females            16    16           16
80  ontogeny_females             8     8            8
    ontogeny_males              16    16           16
84  ontogeny_females            16    16           16
    ontogeny_males              16    16           16
90  dana_ontogeny_males         16    16           16
    ontogeny_females             8     8            8
103 ontogeny_males              11    11           11
127 ontogeny_females             8     8            8

In [28]:
# sub-select 4 sessions from each group
# 64 sessions = 3.5e6 frames
sample = df.groupby(['age', 'experiment']).sample(n=3, random_state=0)

In [29]:
len(sample)

153

In [30]:
len(sample) * 30 * 60 * 20

5508000

In [31]:
for path in map(Path, sample['path']):
    try:
        new_path = agg_folder / (path.parents[1].name + '.h5')
        new_path.symlink_to(path)
        new_path = agg_folder / (path.parents[1].name + '.yaml')
        new_path.symlink_to(path.with_suffix('.yaml'))
    except FileExistsError:
        print(path, 'exists in training set')